## How to initalize
Suppose we have a polynomial of the form:

$a_0 + a_1x + a_2x^2 + ... a_nx^n$

We want to initalize it like:

`a = Polynomial(`$a_0$, $a_1$, $a_2$, ..., $a_n$`)`

In [32]:
class Polynomial(list):
    def __init__(self, *args):
        super(Polynomial, self).__init__(*args)
        
    def __repr__(self):
        text = ""
        
        pow = 0 # Counter to keep track of what power we are on
        for term in self:
            if pow == 0:
                # For the constant term, don't print out x^0
                text += str(term)
            else:
                text += "{0}x^{1}".format(term, pow)
            
            # Add plus sign after term if it's not the last term
            if pow + 1 < len(self):
                pow += 1
                text += " + "
            
        return text

In [30]:
x = Polynomial([1, 2, 3])
x

1 + 2x^1 + 3x^2

In [7]:
class Polynomial(list):
    def __init__(self, *args):
        super(Polynomial, self).__init__(*args)
    def __add__(self, other):
        new_terms = []
        
        i = 0
        while i < max(len(self), len(other)):
            new_terms.append(self[i] + other[i])
            i += 1

        return Polynomial(new_terms)

In [8]:
x = Polynomial([1,2,3])
x

[1, 2, 3]

In [9]:
y = Polynomial([0, 0, 0, 1])
y

[0, 0, 0, 1]

In [10]:
x + x

[2, 4, 6]

In [11]:
x + y

IndexError: list index out of range

In [12]:
class Polynomial(list):
    def __init__(self, *args):
        super(Polynomial, self).__init__(*args)
    def __add__(self, other):
        new_terms = []
        
        i = 0
        while i < max(len(self), len(other)):
            new_terms.append(self[i] + other[i])
            i += 1

        return Polynomial(new_terms)
    def __getitem__(self, key):
        if key < len(self):
            return super(Polynomial, self).__getitem__(key)
        else:
            return 0

In [13]:
x = Polynomial([0, 0, 0, 1])
y = Polynomial([0.5, 0.5, 0.5])
x + y

[0.5, 0.5, 0.5, 1]

In [14]:
x - y

TypeError: unsupported operand type(s) for -: 'Polynomial' and 'Polynomial'

In [17]:
class Polynomial(list):
    def __init__(self, *args):
        super(Polynomial, self).__init__(*args)
    def __add__(self, other):
        new_terms = []
        
        i = 0
        while i < max(len(self), len(other)):
            new_terms.append(self[i] + other[i])
            i += 1

        return Polynomial(new_terms)
    def __neg__(self):
        new_terms = [-term for term in self]
        return Polynomial(new_terms)        
    def __sub__(self, other):
        return self.__add__(-other)
    def __getitem__(self, key):
        if key < len(self):
            return super(Polynomial, self).__getitem__(key)
        else:
            return 0

In [19]:
x = Polynomial([1, 1, 1, 1])
y = Polynomial([1, 1, 1, 2])
x - y

[0, 0, 0, -1]